In [49]:
# Note that this code uses libraries from the TENNLab Neuromorphic Framework
import eons
import neuro
import risp
from torchvision import datasets, transforms
import random

# some constants
NUM_INPUTS = 784
NUM_OUTPUTS = 10
MOA = neuro.MOA()

In [50]:
# Fashion MNIST dataset
transform = transforms.ToTensor() # scale down to 0-1

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

In [51]:
print(test_dataset[0][0].shape)

torch.Size([1, 28, 28])


In [53]:
risp_config = { #https://bitbucket.org/neuromorphic-utk/framework/src/93a1bbba2c0a31ca8f036ff2a89a4d6ffae5cec3/processors/risp/README.md#markdown-header-params
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "min_potential": -1,
  "max_delay": 15,
  "discrete": False
}

proc = risp.Processor(risp_config)

# configure eons
eons_param = {
    "starting_nodes": 4,
    "starting_edges": 10,
    "merge_rate": 0,
    "population_size": 1000,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.05,
    "num_mutations": 5,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.65 , "Delay": 0.35 },
    "num_best" : 2
}
eons_inst = eons.EONS(eons_param)

In [54]:
# create the network from which eons will base the others
net = neuro.Network()
net.set_properties(proc.get_network_properties())

for i in range(NUM_INPUTS):
    node = net.add_node(i)
    node.set("Threshold", 1)
    net.randomize_node_properties(MOA, node)
    net.add_input(i)

for i in range(NUM_INPUTS, NUM_INPUTS+NUM_OUTPUTS):
    

{
  "Associated_Data": {},
  "Edges": [],
  "Inputs": [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    102,
    103,
    104,
    105,
    106,
    107,
    108,
    109,
    110,
    111,
    112,
    113,
    114,
    115,
    116,
 